# Neural networks for text classification

This demo provides very basic Numpy implementations of the following:

1. A feed forward NN represented as a doubly linked list
2. A text classifier that uses a CBOW feature representation
3. Autograd

*Note: This demo conveys some of the underlying constructs used in deep learning packages, not their implementations of those constructs.*

### Load DBPedia14 dataset

In [1]:
import datasets
import numpy as np
import pandas as pd

M = 50000

df = datasets.load_dataset(
    'dbpedia_14', 
    split=['train[:100%]', 
           'test[100%:]']
)[0].to_pandas().sample(frac=1).reset_index(drop=True)[:M]

K = len(set(df.label))

/home/chris/anaconda3/envs/ml-base/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset dbpedia_14 (/home/chris/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c)
100%|██████████| 2/2 [00:00<00:00, 501.59it/s]


### GPU acceleration

In [2]:
device = 'cuda' # "cuda|cpu"

if device == 'cuda':
    import cupy as cp
else:
    cp = np

/home/chris/anaconda3/envs/ml-base/lib/python3.9/site-packages/cupy/_environment.py:438: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


### Spacy preprocessing pipeline

In [3]:
import re
from tqdm import tqdm
from typing import List
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')


email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)
    *|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]
    |\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9]
    (?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}
    (?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:
    (?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

replace = [
    ("<[^>]*>", " "),
    (email_re, " "),                           # Matches emails
    (r"(?<=\d),(?=\d)", ""),                   # Remove commas in numbers
    (r"[*\^\.$&@<>,\-/+{|}=?#:;'\"\[\]]", ""), # Punctuation and other junk
    (r"[\n\t\r]", " "),                        # Removes newlines, tabs, creturn
    (r"[^\x00-\x7F]+", ""),                    # Removes non-ascii chars
    (r"\\+", " "),                             # Removes double-backslashs
    (r"\s+n\s+", " "),                         # 'n' leftover from \\n
    (r"\s+", " "),                             # Strips extra whitespace
        (r"\d+", "DIGIT")                      # Map digits to special token <numbr>
]


def transform(doc: str):
    for repl in replace:
        doc = re.sub(repl[0], repl[1], doc.lower())
    doc = " ".join([w for w in doc.split(" ") if not w in sw_nltk])
    return doc

[nltk_data] Downloading package stopwords to /home/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Process data

In [4]:
from tqdm import tqdm

print(df.content[300])

with tqdm(total=M) as bar:
    for i, content in enumerate(df.content.tolist()):
        df.content[i] = transform(content)
        bar.update(1)

print(df.content[300])

 J.R. Tolkien is a gaff-topsail schooner of Netherlands registry used for passenger cruises on the Baltic Sea and elsewhere in European waters.Originally named Dierkow the vessel was built in 1964 as a seagoing diesel-electric tug at the Edgar-André-Werft in Magdeburg East Germany.


  0%|          | 0/50000 [00:00<?, ?it/s]<ipython-input-4-e3b85edb49ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.content[i] = transform(content)
100%|██████████| 50000/50000 [00:10<00:00, 4880.20it/s]

 jr tolkien gafftopsail schooner netherlands registry used passenger cruises baltic sea elsewhere european watersoriginally named dierkow vessel built DIGIT seagoing dieselelectric tug edgarandrwerft magdeburg east germany


### Featurizer

Below we will build a BOW featurizer, and use it as both a CBOW featurizer and as an embedding lookup

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vocab_size = len(set(" ".join(df.content.tolist()).split(" ")))
print(vocab_size)
featurizer = CountVectorizer(max_features=vocab_size, min_df=4, stop_words=None)
featurizer.fit(df.content.tolist());
featurizer.get_idx = {word: idx for idx, word in enumerate(featurizer.get_feature_names())}
featurizer.get_word = {idx: word for idx, word in enumerate(featurizer.get_feature_names())}

N = len(featurizer.get_idx)
N

156905


/home/chris/anaconda3/envs/ml-base/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


24077

### Features

In [6]:
def get_features(dataframe):
    # Labels
    Y = dataframe.label.to_numpy()

    # One-hot features (just word index pointers)
    X = [[featurizer.get_idx[word] for word in doc.split(" ") if featurizer.get_idx.get(word) is not None] 
          for doc in dataframe.content]

    # BOW features
    X_bow = np.zeros(shape=[len(dataframe), N], dtype=int)
    for i, seq in enumerate(X):
        for idx in seq:
            X_bow[i, idx] += 1
    return X_bow, Y

### Data spit

In [7]:
M_tr = int(0.8 * M)
M_cv = int(0.1 * M)
M_te = M - M_tr - M_cv

X_tr, Y_tr = get_features(df[:M_tr])
X_cv, Y_cv = get_features(df[M_tr: M_tr + M_cv])
X_te, Y_te = get_features(df[-M_te:])

del df

In [8]:
[featurizer.get_word[idx] for idx, c in enumerate(X_tr[0]) if c]

['also',
 'census',
 'chahardangeh',
 'county',
 'district',
 'families',
 'gol',
 'iran',
 'known',
 'population',
 'province',
 'rural',
 'shahrake',
 'tehran',
 'village']

## Neural network as a graph

Deep learning packages implement neural networks as a graph, where each node represents a layer within the network, and each edge represents a transformation that maps one layer onto the another. The two most important components of any deep learning library are (i) efficient implementations of the various transformations used in deep learning (graph edges) that are optimized for various hardware (x86, ARM, NVIDIA GPUS, TPUs etc..), and (ii) automatic differentiation (autograd) which enables users to use the package constructs (linear transformations, activations, loss functions, optimizers etc.) without ever having to implement (or even think about) backpropogation. Below is a (very rudimentary and fairly inefficient) implementation of the basic computational units that are needed to implement a feed forward network.

*Note: There are two numpy multiplication operations being used here: 1) `@` for matrix multiplication, and 2) `*` for the element-wise (aka Hadamard, aka Schur) product*

### Cross entropy loss

In [9]:
def cross_entropy(P, y):
    ce = cp.sum(-cp.asarray(y) * cp.log(P), axis=1)
    return cp.mean(ce)

### Metrics

In [10]:
def compute_accuracy(Y_hat, Y):
    acc = np.equal(Y_hat, Y).mean()
    return acc

### Softmax

In [11]:
def softmax(Z):
    Z = cp.exp(Z - cp.max(Z, axis=1, keepdims=True))
    partition = cp.sum(Z, axis=1, keepdims=True)
    return Z / partition

### Weight initializer

Weight initialization is crucially important in deep learning, if not done properly your network will just diverge after the first gradient update. There are many options here, but it primarily boils down to what distribution do you want your intial weight values to be drawn from. A great choice is *Xavier* initialization, named after it's author Xavier Glorot, which draws weights from a normal distribution $W_{ij} \sim N(\mu=0, \sigma^2=\frac{1}{n})$, where $W \in \mathbb{R}^{n \times \cdot}$. For large weight matrices, this is a means to control the variance of the output, $z$, at each layer.

In [12]:
def weight_init(*dims):
    """ Xavier initialization """
    return (1 / cp.sqrt(dims[0])) * cp.random.randn(*dims)

### Dropout module

Dropout is one of the most important regularization techniques in deep learning. In each activation layer during training, a random subset (chosen with probability `prob`) of the activations are masked out (zeroed). On an intuitive and somewhat handwavy level, you can think of this as a way to prevent the network over relying on on a small subset of the pathways through the network, and helps force all of the nodes, and therefore weights, to be utilized (quasi) equally. By doing this, we are scaling the values of the output layer; in order to keep the distribution over each layer consistent during the training and test phases, this means we must apply a scaling factor either during test time ($\frac{1}{prob}$, traditional dropout), or during training ($\frac{1}{1-prob}$, inverted dropout). Inverted dropout is the preferred method as it incurs no runtime cost during inference, while only adding marginal runtime overhead during training (remember, the vast majority of the computation during training is from backpropogation). This is an implementation of inverted dropout.

In [13]:
def apply_dropout(a: cp.ndarray, prob: float):
    mask = cp.random.choice([0.0, 1.0], size=a.shape, p=[prob, 1 - prob])
    a *= mask / (1 - prob)
    return a

### Graph node

The doubly linked list is a natural data structure for implementing networks. Below we make a class called `Node` which will serve as the base class for the various layer types in our network. In this implementation, each `Node` will contain three things: (i) memory to store a layer (`Node.out`), (ii) a method to map the parent node's output layer to its output (`Node.out = Node.forward(Node.last.out)`), (iii) a method to backpropogate gradients from its child node to its parent node (`Node.error = Node.backward(Node.next.error)`), and (iv) a set of methods to perform gradient updates to the parameters that it owns (`Node.update()`, `Node.zero_grad()`).

In [14]:
class Node:
    
    """ 
    Node in computational graph (doubly linked list)
    Note: not to be confused with a node in a network layer
    
    Attributes:
    last: Node        # Parent
    next: Node        # Child
    out: np.ndarray   # Output layer
    error: np.ndarray # dCE/d`out`
    dim: int          # layer dimension
    lr: float         # learning rate
    """
    
    def __init__(self, last, dim: int, lr: float = None):
        self.last = last
        if self.last:
            self.last.next = self
        self.next = None
        self.out = None
        self.error = None
        self.dim = dim
        self.lr = lr
        
    def forward(self, dropout: bool = False):
        pass
    
    def backward(self):
        pass
    
    def zero_grad(self):
        pass
    
    def update(self):
        pass

### Input embedding node

This is an embedding lookup, which will be the first layer in our network. The forward method accepts a batch of BOW features and computes their inner-product with an embedding lookup table. This is an implementation of the continuous *bag-of-features*, in which we we simply sum the embedding for each word in the the BOW input, weighted by their frequency counts.

In [15]:
class InputEmbedding(Node):
    
    def __init__(self, vocab_size, embedding_dim, lr, wt_decay):
        super().__init__(None, embedding_dim, lr)
        self.W = weight_init(embedding_dim, vocab_size)
        self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay
        self.zero_grad()
    
    def forward(self, X, dropout):
        self.input = cp.asarray(X)
        self.out = self.input @ self.W.T + self.b
        if self.next:
            self.next.forward(dropout)
    
    def zero_grad(self):
        self.W_grad = np.zeros_like(self.W)
        self.b_grad = np.zeros_like(self.b)
    
    def update(self):
        n = self.out.shape[0]
        l1_grad = self.wt_decay * np.sign(self.W)
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l1_grad + l2_grad)
        self.b -= self.lr * (1 / n) * self.b_grad
    
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.input
        db = 1
        self.W_grad += error.T @ dW
        self.b_grad += cp.sum(error, axis=0) * db

### Leaky relu activation

The leaky relu (rectified linear unit) is a popular activation function (non-linearity).

$LReLU(z) = \max(z, \alpha z) \quad \text{where} \quad \alpha \in (0, 1)$

In [16]:
class LReLU(Node):
    
    def __init__(self, last, alpha=0.001, dropout=0.5):
        super().__init__(last, last.dim)
        self.alpha = alpha
        self.dropout = dropout
    
    def forward(self, dropout):
        self.out = cp.where(self.last.out >= 0, 
                            self.last.out, 
                            self.alpha * self.last.out)
        if dropout:
            self.out = apply_dropout(self.out, prob=self.dropout)
        self.next.forward(dropout)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()
        
    def backward(self):
        da = cp.where(self.last.out >= 0, 1, self.alpha)
        self.error = self.next.error * da
        self.last.backward()

### Note on the *forward* and *backward* pass

In the above cell you'll note that the `.forward()` method contains a call to `Node.next.forward()`; this is what allows data to flow from input layer to output layer. Also notice that the `.backward()`, `.update()`, and `.zero_grad()` methods all contain calls to either/both its child (`Node.next`) or parent (`Node.last`) along with code that computes gradents and updates parameter weights; this is very basic implementation of *autograd*. These features allow us to simply call `head_node.forward(some_input)` to compute the *forward pass*, and `tail_node.backward()` to compute the *backward pass* in the training loop at the bottom of this notebook.

### Linear node

This is the *linear* unit, sometimes referred to as a `dense` layer. It's just a batched implementation of $XW^T + b$.

In [17]:
class Linear(Node):
    
    def __init__(self, last, dim: int, lr: float, wt_decay: float, has_bias: bool = True):
        super().__init__(last, dim, lr)
        self.has_bias = has_bias
        self.W = weight_init(self.dim, self.last.dim)
        if has_bias:        
            self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay
        self.zero_grad()
    
    def zero_grad(self):
        self.W_grad = cp.zeros_like(self.W)
        if self.has_bias:
            self.b_grad = cp.zeros_like(self.b)
        self.error = None
        self.last.zero_grad()
    
    def update(self):
        n = self.out.shape[0]
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l2_grad)
        if self.has_bias:
            self.b -= self.lr * (1 / n) * self.b_grad
        self.last.update()
    
    def forward(self, dropout):
        self.out = self.last.out @ self.W.T
        if self.has_bias:
            self.out += self.b
        if self.next:
            self.next.forward(dropout=dropout)
        
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.last.out
        self.W_grad += error.T @ dW
        if self.has_bias:
            db = 1
            self.b_grad += np.sum(error, axis=0) * db
        self.error = error @ self.W
        self.last.backward()

## Network wrapper

This is a wrapper around our doubly linked list, it allows us to define one object for interacting with the network rather than having to keep track of the *head* and *tail* nodes of our network.

In [18]:
class Network:
    
    input: Node
    last: Node
    probs: np.ndarray
    pred: np.ndarray
    
    def set_lr(self, lr):
        node = self.input
        while node is not None:
            node.lr = lr
            node = node.next
            
    @property
    def lr(self):
        return self.last.lr
        
    @property
    def out(self):
        return self.last.out
    
    def forward(self, x, dropout=False):
        self.input.forward(x, dropout=dropout)
        self.probs = softmax(self.out)
        self.pred = cp.argmax(self.probs, axis=1).get()
    
    def backward(self, Y):
        loss = self.probs - cp.asarray(Y)
        self.last.backward(loss)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()

In [19]:
class ANN(Network):
    def __init__(self, vocab_size, layer_size, output_dim, num_layers, lr, wt_decay, alpha, dropout):
        super().__init__()
        self.input = InputEmbedding(vocab_size, layer_size, lr, wt_decay)
        self.last = LReLU(self.input, alpha=alpha, dropout=dropout)
        for i in range(num_layers - 1):
            z = Linear(self.last, layer_size, lr, wt_decay)
            self.last = LReLU(z, dropout=dropout)
        self.last = Linear(self.last, output_dim, lr, wt_decay, has_bias=False)

### Training hyperparameters

In [20]:
alpha = 0.001
lr_init = 0.008
lr_min = 0.0001
lr_decay = 0.975
wt_decay = 1e-4
dropout = 0.1
layer_size = 400
layers = 2
batch_size = 16
batch_size_inference = 512
epochs = 50

## Training loop

In [21]:
net = ANN(N, layer_size, K, layers, lr=lr_init, wt_decay=wt_decay, alpha=alpha, dropout=dropout)

one_hot_labels = np.eye(K)

shuffle_idx = np.arange(M_tr)

with tqdm(total=epochs * (M_tr // batch_size)) as bar:    
    
    for epoch in range(epochs):
        
        # Randomize data ordering
        np.random.shuffle(shuffle_idx)
        X_shfd = X_tr[shuffle_idx]
        Y_shfd = Y_tr[shuffle_idx]
        
        for i in range(0, M_tr, batch_size):
            
            # Get batch of x,y pairs
            x = X_shfd[i: i + batch_size]
            y = Y_shfd[i: i + batch_size]
            
            # Skip empty slices
            if not x.shape[0]:
                continue
            
            # Forward pass
            net.forward(x, dropout=True)
            
            # Backward pass
            # Note: From lecture 05, remember that the derivative of the
            # cross entropy loss w.r.t. the input (z) to the softmax P(z)
            # is dCE/dz = P(z) - y.
            net.backward(one_hot_labels[y])
            
            # Apply gradient updates
            net.update()
            
            # Zero the stored gradients
            net.zero_grad()
            
            bar.update(1)
        
        # Evaluate performance on training set w/o dropout
        tr_results = []
        for i in range(0, M_tr, batch_size_inference):
            x = X_tr[i: i + batch_size_inference]
            y = Y_tr[i: i + batch_size_inference]
            net.forward(x)
            ce_tr = cross_entropy(net.probs, one_hot_labels[y])
            acc_tr = compute_accuracy(net.pred, y)
            tr_results.append([x.shape[0] / M_tr, ce_tr, acc_tr])
        
        ce_tr = sum([x[0] * x[1] for x in tr_results])
        acc_tr = sum([x[0] * x[2] for x in tr_results])
        
        net.forward(X_cv)
        ce_cv = cross_entropy(net.probs, one_hot_labels[Y_cv])
        acc_cv = compute_accuracy(net.pred, Y_cv)

        bar.set_description("Epoch: %d, lr: %.5f, CE (train/val): %.4f / %.4f, Acc (train/val): %.4f / %.4f" 
                            % (epoch + 1, net.lr, ce_tr, ce_cv, acc_tr, acc_cv))

        # Learning rate annealing
        net.set_lr(max(lr_min, lr_decay * net.lr))

Epoch: 50, lr: 0.00231, CE (train/val): 0.0338 / 0.1571, Acc (train/val): 0.9913 / 0.9590: 100%|██████████| 125000/125000 [33:18<00:00, 62.54it/s]  


### Compute test performance

In [23]:
net.forward(X_te)
ce_te = cross_entropy(net.probs, one_hot_labels[Y_te])
acc_te = compute_accuracy(net.pred, Y_te)
print(f"Test CE: {ce_te:.4}, Test Acccuracy: {acc_te}")

Test CE: 0.1724, Test Acccuracy: 0.9548
